# **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note** If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".

---

**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

In [ ]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

In [ ]:
#reading in an image
image = mpimg.imread('test_images/solidWhiteRight.jpg')
#printing out some stats and plotting
print('This image is:', type(image), 'with dimesions:', image.shape)
plt.imshow(image)  # if you wanted to show a single color channel image called 'gray', for example, call as plt.imshow(gray, cmap='gray')

**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images
`cv2.cvtColor()` to grayscale or change color
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

Below are some helper functions to help get you started. They should look familiar from the lesson!

In [ ]:
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 100, 0], thickness=3):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.6, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

## Test on Images

Now you should build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

In [ ]:

import os
os.listdir("test_images/")

run your solution on all test_images and make copies into the test_images directory).

In [ ]:
# modified function definitions go here
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2

import statistics as st
import operator

%matplotlib inline

def distance(a, b):
    """ Distance between two points"""
    return math.hypot(b[0] - a[0], b[1] - a[1])

def line_length(line):
    """ Lenght of a line"""
    return math.hypot((line[0][2]-line[0][0]),(line[0][3]-line[0][1]))

def draw_poly(img, vertices,color=[0, 0, 255], thickness=2):
    """ Draws a closed polygon with the given vertices in blue color. Helper function to identify window """
    
    zero_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    isClosed = 1
    
    cv2.polylines(zero_img, vertices, isClosed, color, thickness)
    return zero_img

def fit_line(lane):
    """ Custom implementation of polyfit. As the data is stored in a list of lists representing Hough lines"""
    #get all x for lane
    x= lane[:,::2].flatten() #selecting only the x elements from the list and flatten it
    #get all y for lane
    y= lane[:,1::2].flatten() #selecting only the y elements from the list and flatten it
    
    #fit to the line, using regression
    coeff = np.polyfit(x, y, 1)
    return coeff

def get_lines_by_side(lines_list, choosen_side, removed_lines):
    """ INPUT: list of all Hough lines and the side on  
        OUTPUT: the remaining hough lines after filtering the ones on different side and different slope. """
    for line in lines_list:
        if (line['side']== choosen_side ) and (line['index'] not in removed_lines) :
            #update the right lane list     
            if 'sel_lines' not in dir():
                sel_lines = np.array((line['line'][:]),)
            else:
                sel_lines = np.vstack((sel_lines,line['line'][:]))
                
#    if 'sel_lines' not in dir():
#        print('choosen_side', choosen_side)
#        print('removed_lines', removed_lines)
#        print(lines_list)
#        sel_lines = np.array((1,4),'i4')
            
    return sel_lines

def get_coordinates(filtered_lane):
    """ INPUT: The best set of points that represent a lane
        OUTPUT: Co-ordinates and a polynomial that represents this lane so that it can be extended """
    #get all x for lane
    x= filtered_lane[:,::2].flatten() #selecting only the x elements from the list and flatten it
    #get all y for lane
    y= filtered_lane[:,1::2].flatten() #selecting only the y elements from the list and flatten it

    #use polyfit to fit to a straight line
    filter_coeff = fit_line(filtered_lane)
    #from the co-efficients, generate the equation for the line, smoothen_y = mx + c
    polynomial = np.poly1d(filter_coeff)

    sorted_x = sorted(x)
    x1 = sorted_x[0]
    x2 = sorted_x[len(sorted_x)-1]
    
    y1 = int(polynomial(x1))
    y2 = int(polynomial(x2))
    
    
    y1 = int(polynomial(x1))
    y2 = int(polynomial(x2))
    
    return (x1,y1,x2,y2,polynomial, filter_coeff)



In [ ]:
# TODO: Build your pipeline that will draw lane lines on the test_images
# then save them to the test_images directory.


#Draws the detected lane in final image
def draw_lane(img, lines, color=[255, 0, 0], thickness=5):
    """ ALGORITHM PIPELINE: 2
        INPUT: List of hough lines OUTPUT: Two lanes drawn on an image"""
        
    lines_list = np.zeros (len(lines), dtype=[('index','i1'), ('line','i4',(1,4)),('slope','f8'),('side','a8'),('len','f4')])
    left_longest = np.zeros (1, dtype=[('index','i1'), ('len','i4')])
    right_longest = np.zeros (1, dtype=[('index','i1'), ('len','i4')])
    
    
    """ 1. Pre-processing1: find slope and length for all the lines in the list """
    
    # Begin of for loop pre-processing input data
    for i in range(len(lines)):
        lines_list[i]['index'] = i
        lines_list[i]['line'] = lines[i]
        #check for infinity or vertical lines
        if (lines[i][:,2] != lines[i][:,0]) :
            #update the slope
            lines_list[i]['slope'] = ((lines[i][:,3]-lines[i][:,1])/(lines[i][:,2]-lines[i][:,0]))
            if 0.1 < lines_list[i]['slope'] < 0.9:
                lines_list[i]['side'] = str("right")
            elif -0.1 > lines_list[i]['slope'] > - 0.9:
                lines_list[i]['side'] = str("left")
            else: #check for horizontal lines
                lines_list[i]['side'] = str("none")
        else:
            lines_list[i]['side'] = str("none")
    
        #find length of each line
        lines_list[i]['len'] = (line_length(lines[i])) 
        
    ## 1. Pre-processing2: find the longest left and right line. 
    ## Separate the lines representing right lane, left lane and on their length
        if (lines_list[i]['side']== b"left") :
            if (lines_list[i]['len'] > left_longest[0]['len'] ):
                left_longest[0]['index'] = i
                left_longest[0]['len'] = lines_list[i]['len']
            #update the left lane list     
            if 'left_lane' not in dir():
                left_lane = np.array((lines_list[i]['line'][:]),)
                left_lane_len = np.array(math.ceil(lines_list[i]['len']))
            else:
                left_lane = np.vstack((left_lane,lines_list[i]['line'][:]))
                left_lane_len = np.append(left_lane_len,math.ceil(lines_list[i]['len']))

                
        elif (lines_list[i]['side']== b"right") :
            #find the longest line
            if (lines_list[i]['len'] > right_longest[0]['len'] ):
                right_longest[0]['index'] = i
                right_longest[0]['len'] = lines_list[i]['len']
                
            #update the right lane list     
            if 'right_lane' not in dir():
                right_lane = np.array((lines_list[i]['line'][:]),)
                right_lane_len = np.array(math.ceil(lines_list[i]['len'])) 
            else:
                right_lane = np.vstack((right_lane,lines_list[i]['line'][:]))
                right_lane_len = np.append(right_lane_len,math.ceil(lines_list[i]['len']))
                
    #### End of For loop, pre-processing the input data
                
    """ 2. Find the average slope of right and left lanes """
    
    #find the average length of the left and right lanes
    mean_len_l = st.mean(left_lane_len)
    mean_len_r = st.mean(right_lane_len)
    
    # Determine the weight of each line for the slope. By dividing the lenght by its mean, 
    # we are preventing a statistical error, where noise from shorter straight lines 
    # can be filtered out
    right_lane_len_weight = [int(ele/mean_len_r) for ele in right_lane_len] 
    left_lane_len_weight = [int(ele/mean_len_l) for ele in left_lane_len] 

    
    #find the slope of all left and right leaning lines
    right_slope = ((right_lane[:,3]-right_lane[:,1])/(right_lane[:,2]-right_lane[:,0]))    
    left_slope = ((left_lane[:,3]-left_lane[:,1])/(left_lane[:,2]-left_lane[:,0]))

    
    #find the average slope, with higher weightage given to lines above mean length
    mean_slope_r = np.average(right_slope, None, right_lane_len_weight)
    mean_slope_l = np.average(left_slope,None, left_lane_len_weight)

    """ 3. Filter and choose the lines within 30% of average slope """
    removed_lines = []
    #filter out of 30% slope, lines
    for line in lines_list:
        if line['side'] == b"right":
            #slope is positive, say not between 0.4 < .5 < 0.6 then remove
            if not (mean_slope_r-0.15 < line['slope'] < mean_slope_r+0.15) :
                removed_lines = np.append(removed_lines,line['index'])   
        if line['side'] == b"left":
            #slope is negative, say not between -0.4 > -.5 > -.6 then remove
            if not (mean_slope_l+0.15 > line['slope'] > mean_slope_l-0.15) :
                removed_lines = np.append(removed_lines,line['index'])   
                                
        
    filtered_right = get_lines_by_side(lines_list, b"right", removed_lines)
    filtered_left = get_lines_by_side(lines_list, b"left", removed_lines)

    
    """ 4. find the co-ordinates of the detected left and right lines """
    
    x1,y1,x2,y2,polynomial_l, coef_l = get_coordinates(filtered_left)

    cv2.line(img, (x1, y1), (x2, y2), color, thickness)
#    print('left lane',x1,y1,x2,y2,polynomial_l)
    
    x1_r,y1_r,x2_r,y2_r,polynomial_r,coef_r = get_coordinates(filtered_right)
        
    cv2.line(img, (x1_r, y1_r), (x2_r, y2_r), color, thickness)
#    print('right lane',x1_r,y1_r,x2_r,y2_r,polynomial_r)
    points = [x1,y1,x2,y2,x1_r,y1_r,x2_r,y2_r]


    """ 5. Extrapolate the left and right lines  """

    #. extend the left lane to the bottom
    x2=x1
    y2=y1
    x1=0
    y1=int(polynomial_l(0))
#    color =[0,255,0]
        
    cv2.line(img, (x1, y1),(x2, y2), color, thickness)
#    print('left lane',x1,y1,x2,y2,polynomial_l)
    
        #. extend the right lane to the bottom
#    print('img shape',img.shape)
    x1_r = x2_r
    y1_r = y2_r
    y2_r = img.shape[0]
    x2_r = int((y2_r - coef_r[1]) / coef_r[0]) 

    cv2.line(img, (x1_r, y1_r), (x2_r, y2_r), color, thickness)
    points = [x1,y1,x2,y2,x1_r,y1_r,x2_r,y2_r]

    return points

def my_hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    points = draw_lane(line_img, lines)
    return line_img, points


def draw_frame(image, imshape):
    """ The Line finding pipeline for each frame """

    #1. Convert to gray scale
    gray_image = grayscale(image)

    #2. smooth the image using Gaussian
    kernel_size = 5
    gaus_image = gaussian_blur(gray_image, kernel_size)

    #3. Identify the edges in the image using canny edge detection
    canny_image = canny(gaus_image, 50,150)


    #4. Mask the image and select the area
    #vertices = np.array([[(x1,y1),(x2,y2), (x3,y3), (x4,y4)]], dtype=np.int32)
    view_depth = 330 #height of the polygon
    vertices = np.array([[(150,imshape[0]),(420, view_depth), (550, view_depth), (900,imshape[0])]], dtype=np.int32)
    masked_image = region_of_interest(canny_image, vertices)

    ### Just for the purpose of presentation
    poly_image = draw_poly(image, vertices)
    weight_image = weighted_img(poly_image, image)

    #5. Then do a hough transform to choose the list of line segments
    rho = 2
    theta = np.pi/180
    threshold = 15 
    min_line_len = 10
    max_line_gap = 30
    hough_line_image, points = my_hough_lines(masked_image, rho, theta, threshold, min_line_len, max_line_gap)



    #6. Draw the line segments on the original image

    weight_hough_image = weighted_img(hough_line_image, image)

    #plt.figure()
    #plt.title("Gray Image")
    #plt.imshow(gray_image, cmap='gray')

    #plt.figure()
    #plt.title("canny Image")
    #plt.imshow(canny_image, cmap='gray')

    #plt.figure()
    #plt.title("masked image")
    #plt.imshow(masked_image, cmap='gray')

    #plt.figure()
    #plt.title("poly image")
    #plt.imshow(poly_image, cmap='gray')

    #plt.figure()
    #plt.title("weighted image")
    #plt.imshow(weight_image)
#    print('\n points: left: ',points[0],',',points[1],',',points[2],',',points[3],)
#    print('points: right: ',points[4],',',points[5],',',points[6],',',points[7])

#    plt.figure()
#    plt.title(image_name)
#    plt.imshow(weight_hough_image)
    
    return weight_hough_image


In [ ]:
images = os.listdir("test_images/")
print(images)
#images = ['solidYellowLeft.jpg']

for image_name in images :
    #reading in an image
    print('-----------> working on ',image_name)
    image = mpimg.imread('test_images/'+image_name)
    weight_hough_image = draw_frame(image,image.shape)
    #plt.figure()
    #plt.title(image_name)
    #plt.imshow(weight_hough_image)
    cv2.imwrite('test_images/processed_'+image_name,weight_hough_image)



## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:


def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image with lines are drawn on lanes)
#    print(type(image))
#    print(len(image))
    

    return draw_frame(image, [540, 960])


Let's try the one with the solid white lane on the right first ...

In [ ]:
white_output = 'white.mp4'
clip = VideoFileClip("solidWhiteRight.mp4")
#print('clip dimensions' ,clip1.size )
#myframe = clip1.get_frame(9)
#newImage = process_image(myframe)

#white_clip = clip1.fl_image(newImage) #NOTE: this function expects color images!!
#%time white_clip.write_videofile(white_output, audio=False)
#%time newImage.write_videofile(white_output, audio=False)

clip_blurred = clip.fl_image( process_image )
%time clip_blurred.write_videofile(white_output,audio=False)


Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

**At this point, if you were successful you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform.  Modify your draw_lines function accordingly and try re-running your pipeline.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [ ]:
yellow_output = 'yellow.mp4'
clip2 = VideoFileClip('solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

#white_output = 'white.mp4'
#clip = VideoFileClip("solidWhiteRight.mp4")
#clip_blurred = clip.fl_image( process_image )
#%time clip_blurred.write_videofile(white_output,audio=False)


In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Reflections

Congratulations on finding the lane lines!  As the final step in this project, we would like you to share your thoughts on your lane finding pipeline... specifically, how could you imagine making your algorithm better / more robust?  Where will your current algorithm be likely to fail?

Please add your thoughts below,  and if you're up for making your pipeline more robust, be sure to scroll down and check out the optional challenge video below!

## My Reflections:
This is a great concept for the project idea. Before starting this project, the only knowledge I had on python is to print a line. Going through the entire image processing pipeline was just awesome. Large amount of time was taken in implementing the ideas in python.

Below are different options tried out, that were a failure,

1. Tried FFT for list of lines, hoping to find the most prominent line slope. But could not successfully find the amplitude of signal with highest frequency. Eventually resorted to weighted average of slopes to filter out the most prominent line

2. Without weighted avergae of the slopes, just taking the slopes to filter out the lines would result in a statistical error, where most number of lines were staright but representing other objects in the plane. other interesting consequence of the weighted average method,
   a. When taking a curve, the lines in the nearer field would get filtered out, due to a greater variation in slope. so, extrapolation needs to be done towards the bottom
  
3. Extrapolating the line - remember the origin is in top left of the image. so,
   a. visualing increasing slope, the left lane is negative instead of positive
   b. While extrapolating the left line x1,y1 needs to be extended down on the left. while x4,y4 needs to be exteded down on the left
   
## Limitations of the algorithm
The algortihtm uses a hard coded value for the view width to determing the masking filter. Hence, it would not work on the challenge problem as it is. 


## Feedback for next iteration of the course
For someone, who is completely new to python. Initially most time was spent in getting used the python dimensions for an array, initializing it, updating and removing elements to a 2D array. having few sample exercises on this topics wpuld speed up for future students of this course

    


## Submission

If you're satisfied with your video outputs it's time to submit!  Submit this ipython notebook for review.


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [ ]:
challenge_output = 'extra.mp4'
clip2 = VideoFileClip('challenge.mp4')
challenge_clip = clip2.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))